# Mongo DB

### Objectives:
* Install `mongo` and `pymongo`. 
* Compare and contrast SQL and noSQL.
* Connect to a mongo server with `mongo` and `pymongo`.
* Perform basic operations using `mongo` and `pymongo`.

## Resources

* [The Little MongoDB Book](http://openmymind.net/mongodb.pdf)
* [PyMongo tutorial](http://api.mongodb.org/python/current/tutorial.html)

## Installing Mongo and PyMongo

### Mongo
1. Create and start a container running the mongodb server,: `$ docker run --name mongoserver -p 27017:27017 -v "$PWD":/home/data -d mongo` (you should be in or above the lecture directory)
2. Access the mongo terminal in the container: `$ docker exec -it mongoserver bash`

**Question:** What do all those options mean?

We'll get to importing the data (with `mongoimport`) shortly.

Some other things you might do with the container:
1. Start the container: `$ docker start mongoserver` (it already is)
2. List all the containers you have, including those stopped `$ docker container ls -a`
3. Stop the container: `$ docker stop mongoserver`
4. Remove the stopped container: `$ docker stop mongoserver`

#### Do *not* run services as `root`.

### PyMongo
You'll need `pymongo` on your computer (not the container) to connect to the mongo server from python.
1. Install PyMongo: `conda install pymongo`

### SQL vs NoSQL

NoSQL does not stand for 'No SQL'. SQL is useful for many things, it's not going away.

> NoSQL ==> "Not Only SQL"

It's a different paradigm to deal with messy data that does not lend itself to an RDBMS.  It's also very useful as a quick and painless solution to data storage, where a full relational database model takes much thought and investment.

In general, if your priority is that all data is in your database is consistent and clean at all times, use a RDBMS. If your priority is preserving all your data as easily and completely as possible, MongoDB might be a better choice.

### Mongo Clients

The command line program we use to interact with mongo is a *client*.  It's only job is to send messages to another program, a *server*, which holds all our data and knows how to operate on it.

The command line Mongo client is written in javascript, so interacting with mongo with this client looks like writing javascript code.

<img src="images/client-server.png" width = 500>

There are other clients.  Late on we will use `pymongo` to interact with our databases from python.

## Javascript Object Notation

Javascript Object Notation, or JSON, is a simple data storage and communication protocall.  It was designed by [Douglas Crockford](https://en.wikipedia.org/wiki/Douglas_Crockford) based on the notation Javascipt uses for objects.

It is meant as a replacement to XML.

```javascript
{
    name: 'TwilightSparkle',
    friends: ['Applejack', 'Fluttershy'],
    age: 16,
    gender: 'f',
    wings: true,
    horn: true,
    residence: {
        town: 'Ponyville',
        address: '15 Gandolfini Lane'}
}
```

It is very similar to a python dictionary literal.

## Working with Mongo DB

### MongoDB Concepts

#### What's it about? 

* MongoDB is a document-oriented database, an alternative to RDBMS, used for storing semi-structured data.
* JSON-like objects form the data model, rather than RDBMS tables.
* Schema is optional.
* Sub-optimal for complicated queries.

#### Structure of the database.

* MongoDB is made up of databases which contain collections (tables).
* A collection is made up of documents (analogous to rows or records).
* Each document is a JSON object made up of key-value pairs (analogous to columns).


So a RDBMS defines columns at the table level, document oriented database defines its fields at a document level.

The `unicorns.json` file that can be imported into MongoDB. From the bash shell, navigate to the mongo-db lecture directory, and do

```
mongoimport --db mlp --collection unicorns < data/unicorns.json
```
If you're using a docker container, use the bash shell **running in that container** (as above) and first navigate to `/home/data`.

**Note**: You may need to add a switch `--jsonArray`.

Now start `mongo`.

A MongoDB contains a collection of databases, so lets check that the `mlp` database exists.

```
> show dbs
```

To use the `mlp` database, we simply do the following:

```
> use mlp
```

A database is made of `collection`s, which are containers for the actual stored data.  A `collection` would be analagous to a `table` in a classical relational database, but can contain much more flexible data than a table.

```
> db.getCollectionNames()
```

### Inserting Data
```javascript
db.unicorns.insert({
    name: 'Applejack',
    age: 15,
    friends: ['TwilightSparkle', 'Fluttershy'],
    wings: false,
    horn: false
})

db.unicorns.insert({
    name: 'Fluttershy',
    age: 15,
    friends: ['Applejack', 'TwilightSparkle'],
    wings: true,
    horn: false
})
                 
```

## Querying Data

Without any arguments, `find` dumps all the data in the collection

```javascript
db.unicorns.find()
```

`find` is much more flexible.

```javascript
// find by single field
db.unicorns.find({name: 'TwilightSparkle'})

// find by presence of field
db.unicorns.find({friends: {$exists : true}})

// find by value in array
db.unicorns.find({friends: 'TwilightSparkle'})

// To return only certain fields use the optional second argument to `find`.
// This says, return only the names of unicorns who are friends with
// twilight sparkle.
db.unicorns.find({friends: 'TwilightSparkle'}, {name: true})
```

**Exercise**: Try to find all the unicorns with wings.

## Updating Data

```javascript
// Replaces friends array
db.unicorns.update({
    name: 'TwilightSparkle'}, {
    $set: {
        friends: ['Shutterfly', 'Rarity', 'Applejack']}})

// Adds to friends array
db.unicorns.update({
    name: 'Applejack'}, {
    $push: {
        friends: 'Rarity'}})
```

We have to use the `$set` and `$push` operators, the default behaviour of `update` is to *replace the data*.

```javascript
// Replaces the TwighlightSparkle data completely!
// It will no longer have even a name field after this!
db.unicorns.update({
    name: 'TwilightSparkle'}, {
    friends: ['Shutterfly', 'Rarity', 'Applejack']})
```


An `upsert` either creates a document (when it does not already exist) or inserts into an existing document.

```
// Upsert: This one is created
db.unicorns.update({
    name: "Rarity"}, {
    $push: {
        friends: {
            $each: ["TwilightSparkle", "Applejack", "Fluttershy"]}}}, {
    upsert: true})

// Upsert: This one is updated
db.unicorns.update({
    name: "Fluttershy"}, {
    $push: {
        friends: {
            $each: ["Rarity", "PrincessCelestia"]}}}, {
    upsert: true})
```

**Exercise**: Enter a unicorn named RainbowDash into the database who is friends with TwilightSparkle, Rarity, and Applejack.

## Deleting Data

You can also delete data using a query. Do this with care.

```javascript
db.unicorns.remove({name: "Fluttershy"})
```

## PyMongo


`pymongo` allows python to connect to and manipulate MongoDB.

In [ ]:
from pymongo import MongoClient
import pprint

In [ ]:
# Connect to the hosted MongoDB instance
client = MongoClient('localhost', 27017)

In [ ]:
db = client['mlp']

In [ ]:
# Create a collection called unicorn
unicorns = db['unicorns']

In [ ]:
unicorns.insert_one({
    'name': 'RainbowDash', 
    'age': 16, 
    'friends': ['TwilightSparkle', 'Applejack', 'Rarity']})

In [ ]:
unicorns.count_documents({})

In [ ]:
pprint.pprint(unicorns.find_one())

In [ ]:
rarity = unicorns.find_one({'name': 'Rarity'})
pprint.pprint(rarity)

The same selector strategies can be used for more complex queries in `pymongo`

In [ ]:
friend_of_twilight = unicorns.find_one({'friends': 'TwilightSparkle'})
pprint.pprint(friend_of_twilight)

To get multiple results back, use `find`, which returns an iterator.

In [ ]:
friends_of_twilight = unicorns.find({'friends': 'TwilightSparkle'})
for friend in friends_of_twilight:
    pprint.pprint(friend)

In [ ]:
young_unicorns = unicorns.find({'age': {'$lt': 33}})
for unicorn in young_unicorns[:2]:
    pprint.pprint(unicorn)

**Exercise:** Find all the unicorns that have a horn and wings.